In [1]:
import pandas as pd
import os
import json
import re

from bs4 import BeautifulSoup
from unidecode import unidecode

In [2]:
# Pasta onde os arquivos HTML estão armazenados
html_folder = './avisos-documentos'

In [3]:
# Cria uma lista vazia para armazenar os dados de texto
data = {'ID': [], 'rotulo': [], 'texto': [], 'nome_arquivo': []}

In [4]:
# Lista para salvar arquivos que falharam na conversao
files_error = []

In [5]:
# Itera sobre os arquivos HTML na pasta especificada
for filename in os.listdir(html_folder):
    if filename.endswith('.html'):
        try:
            # Abre o arquivo HTML e cria um objeto BeautifulSoup
            with open(os.path.join(html_folder, filename)) as file:
                soup = BeautifulSoup(file, 'html.parser')

            # Extrai o texto do arquivo HTML
            text = soup.get_text()

            # Retirar caracteres especiais
            filename = filename.replace("ç", "")
            filename = filename.replace("ã", "")
            
            # Adiciona os dados ao dicionário
            data['texto'].append(text)
            data['nome_arquivo'].append(filename)
            data['rotulo'].append("Sem rotulo")
            data['ID'].append('temp')
        except Exception as e:
            files_error.append(filename)

In [6]:
df = pd.DataFrame(data)

In [7]:
# Retirar '\n'
df['texto'].replace('\n', '', regex=True, inplace=True)

In [8]:
df.head()

,ID,rotulo,texto,nome_arquivo
0,temp,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...,00174550920008070001_20181126132136_25834400-C...
1,temp,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...,00174550920008070001_20190814152826_41551083-D...
2,temp,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...,00174550920008070001_20190930131835_45409481-S...
3,temp,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...,00174550920008070001_20200608132908_64948369-C...
4,temp,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...,00259102620018070001_20201014133202_74232911-C...


In [9]:
df['nome_arquivo'][0]

'00174550920008070001_20181126132136_25834400-Certido.html'

In [10]:
# Caminho para o arquivo JSON
json_file = './labelbox_annotations.json'

In [11]:
# Carrega o arquivo JSON em um objeto Python
with open(json_file, 'r') as f:
    data = json.loads(f.read())

In [12]:
# Itera sobre os objetos na lista
for obj in data:
    # Extrai as informações desejadas
    file_name = obj['External ID']
    id = obj['ID']

    # Retirar caracteres especiais
    file_name = file_name.replace("Ã§", "")
    file_name = file_name.replace("Ã£", "")

    df.loc[df['nome_arquivo'] == file_name, 'ID'] = id
        
    try:
        label = obj['Label']['classifications'][0]['answer']['title']
        df.loc[df['nome_arquivo'] == file_name, 'rotulo'] = label
    except Exception as e:
        pass

In [13]:
df = df.drop('nome_arquivo', axis=1)

In [14]:
# Quantidade de linhas com o rotulo diferente de -1
len(df.loc[df['rotulo'] != -1])

3967

In [19]:
df.head(10000)

,ID,rotulo,texto
0,clbdwul7j07wv070k436t69qr,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
1,clbdwumaj07x0070kco8701gu,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
2,clbc8u00n441c071v5alx2zm7,Recurso/Dispensa,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
3,clbc8u0pn1u87082igpm586rs,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
4,clbc8y06s2rd507264nu265mf,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
...,...,...,...
3962,clbb676250otw073w0n64e1pe,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
3963,clbb67p0r0p6307581sla0prm,InfEspol_Em,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
3964,clbb67vhl0p3y071p4rblbz2i,Recurso/Dispensa,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...
3965,clbb684j30ow6070z9z33856c,Sem rotulo,Poder Judiciário da UniãoTRIBUNAL DE JUSTIÇA D...


In [17]:
# Salva o DataFrame em um arquivo parquet
df.to_parquet('text_data.parquet', index=False)